# Wegsuche auf der Karte
## Karten
Im Prinzip ist eine Karte nichts anderes als ein Graph mit vielen Knoten. In unserem Fall wird ein Bild generiert, bei dem jedes Pixel ein eigener Knoten ist. Jedes Pixel ist hierbei mit seinen acht Nachbarn verbunden (also oben, unten, links, rechts und die vier Diagonalen). 

### Exkurs: Perlin-Noise
Perlin-Noise ist ein Algorithmus, der eine pseudo-zufällige Funktion generiert, die sich gut für die Erzeugung von natürlichen Strukturen eignet. Der Algorithmus erzeugt eine glatte Funktion, die in einem bestimmten Bereich variiert. Diese Funktion kann dann verwendet werden, um Höhenkarten oder andere natürliche Strukturen zu erzeugen. 

In unserem Fall wird Perlin-Noise verwendet, um eine Karte zu generieren, die wie eine natürliche Landschaft aussieht. Die Karte wird dann in ein Bild umgewandelt, das als Grundlage für den Wegsuchalgorithmus dient. 

Spiele wie Minecraft nutzen eine abgewandelte Version von Perlin-Noise, um die Spielwelt zu generieren.

In [ ]:
import time
from utils import perlin_noise, a_stern, dijkstra, draw_path
import numpy as np
import matplotlib.pyplot as plt

# --- Parameter ---
dim = 50
scale = 5
seed = 0
# -----------------
noise = perlin_noise(dim, dim, scale, seed=seed)
noise-=np.min(noise)
noise/= np.max(noise)

# Visualisierung
plt.figure(figsize=(6, 6))
plt.imshow(noise, cmap='terrain', interpolation='lanczos')
plt.axis('off')
plt.colorbar()
plt.show()

## Dijkstra als Wegsuchalgorithmus auf einer Karte
Im folgenden Beispiel wird ein Wegsuchalgorithmus implementiert, der auf einer Karte arbeitet. Der Algorithmus verwendet Dijkstra, um den kürzesten Weg von einem Startpunkt zu einem Zielpunkt zu finden.

In [ ]:
start = np.array([dim//10,dim//10])
ziel = dim-np.array([dim//10,dim//10])

start = [dim//2,dim//2]

distmat = np.linalg.norm(np.stack(np.meshgrid(np.arange(0,dim), np.arange(0,dim))).transpose(2,1,0)-ziel, axis=-1)
heuristik = np.log(distmat+1e-8)+1
distmat /= distmat[start[0], start[1]]
heuristik = distmat

t_start = time.time()
pfad, costmat, besucht = dijkstra(noise, start, ziel, heuristik)
print("Rechenzeit (in s):",time.time()-t_start)

plt.figure(figsize=[10,4])
plt.subplot(1,3,1)
plt.imshow(noise, cmap="terrain", interpolation="lanczos")
plt.scatter(start[1], start[0], marker="x", c="red")
plt.scatter(ziel[1], ziel[0], marker="x", c="blue")
draw_path(pfad)
plt.title("Karte")
plt.subplot(1,3,2)
plt.imshow(costmat)
plt.title("Kosten")
plt.subplot(1,3,3)
plt.imshow(besucht)
plt.title("Besucht (True/False)")
plt.show()

## A* als Erweiterung von Dijkstra
A* ist eine Erweiterung von Dijkstra, die eine Heuristik verwendet, um den Suchraum zu reduzieren. Der Algorithmus verwendet eine Heuristik, um die Kosten für jeden Knoten zu schätzen und wählt den Knoten mit den niedrigsten geschätzten Kosten aus. Dies führt zu einer schnelleren Suche als Dijkstra, da weniger Knoten untersucht werden müssen.

In [ ]:
#np.random.seed(1)
start = np.array([dim//10,dim//10])
#ziel = dim-np.array([dim//10,dim//10])

start = [dim//2,dim//2]
print(start, ziel)

distmat = np.linalg.norm(np.stack(np.meshgrid(np.arange(0,dim), np.arange(0,dim))).transpose(2,1,0)-ziel, axis=-1)
heuristik = distmat

t_start = time.time()
pfad, costmat, besucht = a_stern(noise, start, ziel, heuristik)
print("Rechenzeit (in s):",time.time()-t_start)

plt.figure(figsize=[10,4])
plt.subplot(1,3,1)
plt.imshow(noise, cmap="terrain", interpolation="lanczos")
plt.scatter(start[1], start[0], marker="x", c="red")
plt.scatter(ziel[1], ziel[0], marker="x", c="blue")
draw_path(pfad)
plt.title("Karte")
plt.subplot(1,3,2)
plt.imshow(costmat)
plt.title("Kosten")
plt.subplot(1,3,3)
plt.imshow(besucht)
plt.title("Besucht (True/False)")
plt.show()